### Import libraries

In [2]:
import xlsxwriter
import pandas as pd

### Load Data

In [3]:
orders = pd.read_csv("data/orders_2016_clean.csv")
orders.sort_values(by=["date","order_id"], inplace=True)
orders["date"] = pd.to_datetime(orders["date"])
orders.reset_index(drop=True, inplace=True)
orders.head()

,order_id,date
0,1,2016-01-01
1,2,2016-01-01
2,4,2016-01-01
3,5,2016-01-01
4,6,2016-01-01


In [4]:
order_details = pd.read_csv("data/order_details_2016_clean.csv")
order_details.sort_values(by=["order_id"], inplace=True)
order_details.reset_index(drop=True, inplace=True)
order_details["pizza_id"]=order_details["pizza_id"].str.replace(r"_[smlx]{1,3}$","",regex=True)
order_details.head()

,order_id,pizza_id,quantity
0,1,hawaiian,1.0
1,2,five_cheese,1.0
2,2,thai_ckn,1.0
3,2,classic_dlx,1.0
4,2,mexicana,1.0


In [5]:
orders = orders.merge(order_details, on="order_id")
orders.head()

,order_id,date,pizza_id,quantity
0,1,2016-01-01,hawaiian,1.0
1,2,2016-01-01,five_cheese,1.0
2,2,2016-01-01,thai_ckn,1.0
3,2,2016-01-01,classic_dlx,1.0
4,2,2016-01-01,mexicana,1.0


## GENERAL REPORT

In [6]:
CELL_WIDTH = 64
CELL_HEIGHT = 20

In [7]:
workbook = xlsxwriter.Workbook('output/report.xlsx')
worksheet = workbook.add_worksheet("General_Report")

text = '''GENERAL REPORT'''
options = {
    'width': CELL_WIDTH*8,
    'height': CELL_HEIGHT*4,
    'font':{
        'bold': True,
        'size': 32,
        'color': 'black'
    },
    'align': {'horizontal': 'center', 'vertical': 'middle'},
    'line': {'color': 'black', 'width': 1, 'dash_type': 'solid'},
}
worksheet.insert_textbox("C2", text, options);

### PLOTS USING EXCEL

### Pizzas sold per month

In [8]:
orders["month"] = orders["date"].dt.month
pizzas_mes=orders.groupby("month")["quantity"].sum()

worksheet.write_row("B9", ["Month", "Pizzas Sold"])

worksheet.write_column("B10", pizzas_mes.index)
worksheet.write_column("C10", pizzas_mes.values)

chart = workbook.add_chart({'type': 'column'})

chart.add_series({  
    'name':       f'={worksheet.name}!$C$9',
    'categories': f'={worksheet.name}!$B$10:$B${len(pizzas_mes)+9}',
    'values':     f'={worksheet.name}!$C$10:$C${len(pizzas_mes)+9}',
})

chart.set_title({'name': 'Pizzas vendidas por mes'})
chart.set_x_axis({'name': 'Mes'})
chart.set_y_axis({'name': 'Pizzas vendidas'})
chart.set_x_axis({'num_font':  {'rotation': 45}})
chart.set_style(10)
worksheet.insert_chart('E8', chart);

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64', name='month')
[2850. 2712. 2934. 2924. 2981. 2775. 3036. 2855. 2664. 2687. 2928. 2679.]


### 10 most sold pizzas

In [9]:
pizzas=orders.groupby("pizza_id")["quantity"].sum()
pizzas.sort_values(ascending=False, inplace=True)
pizzas=pizzas[:10]

worksheet.write_row("B26", ["Pizza", "Quantity"])
worksheet.write_column("B27", pizzas.index)
worksheet.write_column("C27", pizzas.values)

chart = workbook.add_chart({'type': 'column'})

chart.add_series({
    'name':       '=General_Report!$C$26',
    'categories': f'=General_Report!$B$27:$B${len(pizzas)+26}',
    'values':     f'=General_Report!$C$27:$C${len(pizzas)+26}',
})
chart.set_title({'name': '10 most sold pizzas'})
chart.set_x_axis({'name': 'Pizza'})
chart.set_y_axis({'name': 'Pizzas vendidas'})
chart.set_x_axis({'num_font':  {'rotation': 45}})
chart.set_style(10)

worksheet.insert_chart('E25', chart);

### 10 least sold pizzas

In [10]:
pizzas=orders.groupby("pizza_id")["quantity"].sum()
pizzas.sort_values(ascending=True, inplace=True)
pizzas=pizzas[:10]

worksheet.write_row("B43", ["Pizza", "Quantity"])
worksheet.write_column("B44", pizzas.index)
worksheet.write_column("C44", pizzas.values)

chart = workbook.add_chart({'type': 'column'})

chart.add_series({
    'name':       '=General_Report!$C$43',
    'categories': f'=General_Report!$B$44:$B${len(pizzas)+43}',
    'values':     f'=General_Report!$C$44:$C${len(pizzas)+43}',
})

chart.set_title({'name': '10 least sold pizzas'})
chart.set_x_axis({'name': 'Pizza'})
chart.set_y_axis({'name': 'Pizzas vendidas'})
chart.set_x_axis({'num_font':  {'rotation': 45}})
chart.set_style(10)

worksheet.insert_chart('E42', chart);

### PLOTS FROM MATPLOTLIB

Using images from last PDF report.

In [11]:
worksheet = workbook.add_worksheet("Predictions_Report")

text = '''PREDICTIONS REPORT'''
options = {
    'width': CELL_WIDTH*9,
    'height': CELL_HEIGHT*4,
    'font':{
        'bold': True,
        'size': 32,
        'color': 'black'
    },
    'align': {'horizontal': 'center', 'vertical': 'middle'},
    'line': {'color': 'black', 'width': 1, 'dash_type': 'solid'},
}
worksheet.insert_textbox("G2", text, options);

### Predictions

In [12]:
txt="""Here is how the model predicts a given ingredient in may 2016. 
To check if our model is aplicable in our daily basis, we'll see how well it predicts ingredients on days that are not in the training set.
The first 7 days are in the training set, the rest are not.

The confidence interval is given by the mean absolute error of that ingredient.

As we can see, the model fits the data of two weeks in the future pretty well, and then it starts to fail but slightly."""


options = {
    'width': CELL_WIDTH*12,
    'height': CELL_HEIGHT*10,
    'font':{
        'bold': True,
        'size': 12,
        'color': 'black'
    },
    'line': {'color': 'black', 'width': 1, 'dash_type': 'solid'},
}

worksheet.insert_textbox("E9", txt, options)

path="images/daily_bbq_ckn.png"
worksheet.insert_image("B21", path, options);

In [13]:
txt=["Now that we know that our model predicts decently the daily stock of ingredients",
"so we can use it to predict the stock of ingredients for an entire week.",
"First, we'll sum up the daily stock of ingredients for each week.",
"Then, we'll add a margin error so that we can be sure that we have enough ingredients.",
"This margin error is the mean absolute error for each ingredient, and it is weighted by a hyperparameter alpha.",
"Finally, lets define a parameter to make the predictions realistic.",
"Let's say we need to make the order of ingredients for the next week on tuesdays.",
"Then, we'll be missing data from tuesday to monday, which is equivalent to having an antelation of 4 days.",
" ",
"Lets see some examples for a couple ingredients"
]
txt="\n".join(txt)

options = {
    'width': CELL_WIDTH*15,
    'height': CELL_HEIGHT*12,
    'font':{
        'bold': True,
        'size': 12,
        'color': 'black'
    },
    'line': {'color': 'black', 'width': 1, 'dash_type': 'solid'},
}

worksheet.insert_textbox("D45", txt, options)

path1="images/weekly_alf.png"
path2="images/weekly_pep.png"
worksheet.insert_image("B60", path1, options)
worksheet.insert_image("B84", path2, options);

In [14]:
txt=["To conclude, lets see how our model does on average over a year.",
"For that, we'll predict the stock of ingredients for each week of 2016,",
"and then we'll compare the mean of ingredients predicted against the mean of ingredients sold.",
" ",
"As before, we'll have same alpha and antelation parameters.",
]
txt="\n".join(txt)

options = {
    'width': CELL_WIDTH*13,
    'height': CELL_HEIGHT*6,
    'font':{
        'bold': True,
        'size': 12,
        'color': 'black'
    },
    'line': {'color': 'black', 'width': 1, 'dash_type': 'solid'},
}

worksheet.insert_textbox("D110", txt, options)

path="images/weekly_all.png"
worksheet.insert_image("B118", path, options)

workbook.close()